In [5]:
import os
# os.environ['OPENBLAS_NUM_THREADS'] = '1'
import castle
from castle.common import GraphDAG
from castle.metrics import MetricsDAG
from castle.datasets import IIDSimulation, DAG
# from castle.algorithms import PC, DirectLiNGAM, ICALiNGAM, GES, CORL, Notears
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings(action='ignore', category=DeprecationWarning, message="`np.int` is a deprecated alias for the builtin `int`")
filterwarnings(action='ignore', category=DeprecationWarning, message="`np.float` is a deprecated alias for the builtin `float`")



In [7]:
def read_matrix(file_path):
    return  pd.read_csv(file_path).to_numpy()

def save_matrix(data, file_path):
    pd.DataFrame(data).to_csv(file_path, index=False)

FOLDER_PATH  = "./causal_data"

In [9]:
for n in [70]:#[10,40,70,100]:
    p = pow(n,-0.7)
    print("n = ",n)
    n_edges = n*n*p/2
    for sample_size in [1000]:#[1000, 4000, 7000, 10000]:
        print("sample_size = ",sample_size)
        for seed in range(1):
            print("here")
            weighted_random_dag = DAG.erdos_renyi(n_nodes=n, n_edges=n_edges,
                                      weight_range=(0.5, 2.0), seed=seed)
            dataset = IIDSimulation(W=weighted_random_dag, n=sample_size, method='linear',
                        sem_type='gauss')
            true_causal_matrix, X = dataset.B, dataset.X
            actual_num_edge=np.count_nonzero(np.array(weighted_random_dag))
#             file_name = "_{}_{}_{}.csv".format(n,sample_size, seed) 
            file_name = "_{}_{}.csv".format(n,sample_size,) 
            save_matrix(true_causal_matrix, os.path.join(FOLDER_PATH, "causal"+file_name))
            save_matrix(X,  os.path.join(FOLDER_PATH, "sim_data"+file_name))

n =  70
sample_size =  1000
here


In [22]:
# algorithms = {"PC":PC(), "DirectLiNGAM": DirectLiNGAM(),"ICALiNGAM": ICALiNGAM(max_iter=10000) }#max_iter=1000000
algorithms = {"Notears":Notears(),  "GES": GES(),  }
# __all__ = ['ANMNonlinear', 'GES', 'TTPM', 'DirectLiNGAM', 'ICALiNGAM', 'PC', 'Notears', 'DAG_GNN',
#            'NotearsLowRank', 'RL', 'CORL', 'GraNDAG', 'NotearsNonlinear', 'GOLEM', 'MCSL', 'GAE']

# data simulation, simulate true causal dag and train_data.
# structure learning
metrics = ['fdr', 'tpr', 'fpr', 'shd', 'nnz', 'precision', 'recall', 'F1', 'gscore']
file_names = os.listdir(FOLDER_PATH)
file_names = [x for x in file_names if x[:6]=="causal"]
for algo_name in algorithms:
    results = []
    t=time.time()
    i=0
    for file_name in file_names:
#     file_name = "causal_10_1000_0_3.csv"
#     file_name = "causal_10_10000_0_3.csv"
#     print(file_name)
        causal_graph = read_matrix(os.path.join(FOLDER_PATH, file_name))
        sim_data = read_matrix(os.path.join(FOLDER_PATH, file_name.replace("causal", "sim_data")))
        _,n,sample_size, seed,actual_num_edge = file_name[:-4].split("_")
        n,sample_size, seed,actual_num_edge = int(n), int(sample_size), int(seed), int(actual_num_edge)
        if seed!=0 or n==100 or sample_size==10000:
            continue
    #         print(algo_name)
        pc = algorithms[algo_name]
        pc.learn(sim_data)

        # plot predict_dag and true_dag
#         GraphDAG(pc.causal_matrix, causal_graph, 'result')

        # calculate metrics
        mt = MetricsDAG(pc.causal_matrix, causal_graph)
        result = [n,sample_size, seed,actual_num_edge, algo_name] + [mt.metrics[x] for x in metrics]
        results.append(result) 
        i+=1
        if i%3==0:
            print(algo_name,i)
            print(t-time.time())
#         print(algo_name)
#         break
#     break
    pd.DataFrame(results, columns = ["n", "samples", "seed", "num_edges", "algo"]+metrics).to_csv("results_"+algo_name+".csv", index=False)
#     i+=1
#     print(i, file_name)
# results = pd.DataFrame(results, columns = ["n", "samples", "seed", "num_edges", "algo"]+metrics)
# results.to_csv("results.csv", index=False)

2022-12-03 00:52:02,367 - /home/aarushi/anaconda3/lib/python3.7/site-packages/castle/algorithms/gradient/notears/linear.py[line:196] - INFO: [start]: n=1000, d=10, iter_=100, h_=1e-08, rho_=1e+16
2022-12-03 00:52:02,529 - /home/aarushi/anaconda3/lib/python3.7/site-packages/castle/algorithms/gradient/notears/linear.py[line:208] - INFO: [iter 0] h=1.707e-01, loss=7.613, rho=1.0e+00
2022-12-03 00:52:02,618 - /home/aarushi/anaconda3/lib/python3.7/site-packages/castle/algorithms/gradient/notears/linear.py[line:208] - INFO: [iter 1] h=1.365e-01, loss=5.051, rho=1.0e+00
2022-12-03 00:52:02,743 - /home/aarushi/anaconda3/lib/python3.7/site-packages/castle/algorithms/gradient/notears/linear.py[line:208] - INFO: [iter 1] h=7.413e-02, loss=5.182, rho=1.0e+01
2022-12-03 00:52:02,935 - /home/aarushi/anaconda3/lib/python3.7/site-packages/castle/algorithms/gradient/notears/linear.py[line:208] - INFO: [iter 1] h=2.743e-02, loss=6.492, rho=1.0e+02
2022-12-03 00:52:03,043 - /home/aarushi/anaconda3/lib/py

KeyboardInterrupt: 

In [4]:
import sys
print(sys.version)


3.9.13 (main, Aug 25 2022, 23:26:10) 
[GCC 11.2.0]


In [2]:
import sys
print(sys.version)


3.9.13 (main, Aug 25 2022, 23:26:10) 
[GCC 11.2.0]


In [1]:
import castle


In [2]:
castle


<module 'castle' from '/home/aarushi/.local/lib/python3.7/site-packages/castle/__init__.py'>